<a href="https://colab.research.google.com/github/jagrajs/Cryptocurrency_Lottery/blob/master/Cryptocurrency_Lottery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cryptocurrency Lottery

by Jagraj Singh

### Instructions

To play:

1.   Sign in with your Google account (required to run cells).
2.   Click 'Runtime' > 'Run all' from the toolbar above.
3.   Expand the 'Play the Ethereum Lottery' section below.
4.   Enjoy!



### Setup

In [0]:
!pip3 install pycryptodome
!pip3 install ecdsa


### Ethereum Lottery

In [0]:
import ast
import codecs
import ecdsa
import hashlib
from Crypto.Hash import keccak
import requests


def private_key_from_string(msg):
  private_key = hashlib.sha256(msg.encode()).hexdigest()
  return(private_key)

def public_key_from_private_key(private_key):
  private_key_bytes = codecs.decode(private_key, 'hex')
  key = ecdsa.SigningKey.from_string(private_key_bytes, curve=ecdsa.SECP256k1).verifying_key
  key_bytes = key.to_string()
  public_key = codecs.encode(key_bytes, 'hex')
  return public_key

def address_from_public_key(public_key):
  public_key_bytes = codecs.decode(public_key, 'hex')
  keccak_hash = keccak.new(digest_bits=256)
  keccak_hash.update(public_key_bytes)
  keccak_digest = keccak_hash.hexdigest()
  # Take the last 20 bytes
  wallet_len = 40
  wallet = '0x' + keccak_digest[-wallet_len:]
  
  # Add checksum
  checksum = '0x'
  address = wallet[2:]
  address_byte_array = address.encode('utf-8')
  keccak_hash = keccak.new(digest_bits=256)
  keccak_hash.update(address_byte_array)
  keccak_digest = keccak_hash.hexdigest()
  for i in range(len(address)):
      address_char = address[i]
      keccak_char = keccak_digest[i]
      if int(keccak_char, 16) >= 8:
          checksum += address_char.upper()
      else:
          checksum += str(address_char)
  return checksum

def get_balance(url):
  try:
    response = requests.get(url)
    status = response.status_code
    if response.status_code == 200:
      response = response.text
      response = ast.literal_eval(response)
      balance = int(response['result'])
      if balance == 0:
        print('\nSorry, you did not win the Ethereum Lottery.')
      else:
        balance = balance / 10**18
        print('Congratulations! You won ' + str(balance) + ' eth!!!\nCopy the private key below!')
    else:
      print('An error has occurred.')
  except:
    print('An error has occurred.')
    
def play():
  msg = input('Welcome to the Ethereum Lottery!\nYour chance of winning is about 1 in 10^70 (essentially zero), but don\'t let that stop you!\nTo play, simply enter some text and hit Enter: ')
  private_key = private_key_from_string(msg)
  public_key = public_key_from_private_key(private_key)
  address = address_from_public_key(public_key)
  api_key = '7Z99TEIBCGR4H7ACKNBU41IUY9QJGTCA82'
  url = 'https://api.etherscan.io/api?module=account&action=balance&address=' + address + '&tag=latest&apikey=' + api_key
  get_balance(url)
  print('\nWallet Information\nAddress: ' + address + '\nPrivate Key: ' + private_key)


### Play the Ethereum Lottery

In [0]:
play()